EnbPI Bootstrap Steps:
1) Extract and Clean Data
2) Preprocess Data
3) Bootstrap Synthetic Data
4) Train Bootstrap Models
5) LOO Predict and Mean Aggregate
6) Get Conformal Prediction Interval 
7) Get Coverage Rate on Test
8) Output Results

In [1]:
import numpy as np
import pandas as pd
from numpy.random import default_rng
from typing import Optional

from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from scipy import stats
from statsmodels.tsa.seasonal import STL
from scipy.special import boxcox, inv_boxcox
from sklearn.preprocessing import PowerTransformer

In [2]:
# Helper functions

# Step 1) Extract and Clean Data
def read_data(data_name, max_data_size):
    data_dir = "./Data"

    if data_name == "Solar_Atl":
        """
        4 (Alternative). NREL Solar data at Atlanta Downtown in 2018.
            - 24 observations per day and separately equally by 1H @ half an hour mark everytime
        Data descriptions see Solar Writeup
        Data download:
        (With API) https://nsrdb.nrel.gov/data-sets/api-instructions.html
        (Manual) https://maps.nrel.gov/nsrdb-viewer
        Properties:
            - Type: multivariate (5 temporal columns, 7 feature columns, 1 target)
            - Length: 8760
            - Period: entire 2018
            - Frequency: hourly (at every 30-min mark)
        """
        file_name = f"{data_name}_data.csv"
        data_df = pd.read_csv(file_name, skiprows=2)
        # data_df.shape  # 8760, 14
        data_df.drop(columns=data_df.columns[0:5], inplace=True)  # remove 5 temporal columns
        data_df.drop(columns="Unnamed: 13", inplace=True)
        # data_df.shape  # 8760, 8
        target_column = "DHI"

    elif data_name in _get_cali_cities():  # "Palo_Alto"
        """
        Properties:
            - Type: multivariate (1 datetime index, 5 temporal columns, 7 feature columns, 1 target)
            - Length: 8760
            - Period: entire 2018
            - Frequency: hourly
        """
        file_name = f"{data_name}_data.csv"
        data_df = pd.read_csv(file_name)
        data_df.drop(columns=data_df.columns[0:6], inplace=True)  # remove datetime index and 5 temporal columns
        target_column = "DHI"

    elif data_name == "appliances":
        """
        2. Appliances energy prediction Data Set
        The data set is at 10 min for about 4.5 months.
        The column named 'Appliances' is the response. Other columns are predictors
        https://archive.ics.uci.edu/ml/datasets/Appliances+energy+prediction
        Properties:
            - Type: multivariate (1 temporal column, 27 feature columns, 1 target)
            - Length: 19735
            - Period: 2016-01-11 @5PM to 2016-05-27 @6PM
            - Frequency: every 10-min
        """
        file_name = f"{data_name}_data.csv"
        data_df = pd.read_csv(file_name, delimiter=",")
        # data_df.shape  # (19736, 29)
        data_df.drop("date", inplace=True, axis=1)  # remove the temporal column
        data_df.loc[:, data_df.columns != "Appliances"]
        target_column = "Appliances"

    elif data_name == "green_house":
        """
        All datasets are Multivariate time-series. They have respective Github for more details as well.
        1. Greenhouse Gas Observing Network Data Set
        Time from 5.10-7.31, 2010, with 4 samples everyday, 6 hours apart between data poits.
        Goal is to "use inverse methods to determine the optimal values of the weights in the weighted sum of 15 tracers that best matches the synthetic observations"
        In other words, find weights so that first 15 tracers will be as close to the last as possible.
        Note, data at many other grid cells are available. Others are in Downloads/🌟AISTATS Data/Greenhouse Data
        https://archive.ics.uci.edu/ml/datasets/Greenhouse+Gas+Observing+Network
        Properties:
            - Type: multivariate (15 feature columns aka tracers(?), 1 target)
            - Length: 327
            - Period: 2010-05-10 to 2010-07-31
            - Frequency: every 6-hours
        """
        file_name = f"{data_name}_data.csv"
        data_df = pd.read_csv(file_name, header=None, sep=" ").T
        # data_df.shape  # 327, 16 Note, rows are 16 time series (first 15 from tracers, last from synthetic).
        target_column = 15

    elif data_name == "Beijing_air":
        """
        3. Beijing Multi-Site Air-Quality Data Data Set
        This data set includes hourly air pollutants data from 12 nationally-controlled air-quality monitoring sites.
        Time period from 3.1, 2013 to 2.28, 2017.
        PM2.5 or PM10 would be the response.
        https://archive.ics.uci.edu/ml/datasets/Beijing+Multi-Site+Air-Quality+Data
        Properties:
            - Type: multivariate (1 index, 4 temporal columns, 12 feature columns, 1 target)
            - Length: 35064
            - Period: 2013-03-01 to 2017-02-28
            - Frequency: hourly
        """
        file_name = f"{data_name}_Tiantan_data.csv"
        data_df = pd.read_csv(file_name)
        # data_df.shape  # 35064, 18
        data_df.drop(
            columns=["No", "year", "month", "day", "hour", "wd", "station"],
            inplace=True,
            axis=1,
        )
        data_df.dropna(inplace=True)
        # data_df.shape  # 32907, 11
        target_column = "PM2.5"

    else:
        raise ValueError("data_name not found")   


    # pick maximum of X data points (for speed)
    data_df = data_df.iloc[: min(max_data_size, data_df.shape[0]), :]
    # shift target column back by one (for single-step forecast)
    data_df[target_column] = data_df[target_column].shift(-1)
    # remove final row as it does not contain target column value anymore
    data_df = data_df.iloc[:-1 , :]        

    return data_df, target_column


def _get_cali_cities():
    cali_cities = [
        "Fremont",
        "Milpitas",
        "Mountain_View",
        "North_San_Jose",
        "Palo_Alto",
        "Redwood_City",
        "San_Mateo",
        "Santa_Clara",
        "Sunnyvale",
    ]
    return cali_cities


# Step 2) this is the AR-transformer. can replace with something from sktime.  ################
def one_dimen_transform(y_train, y_test, n_lags):
    n = len(y_train)
    n1 = len(y_test)
    X_train = np.zeros((n - n_lags, n_lags))  # from n_lags+1,...,n
    X_test = np.zeros((n1, n_lags))  # from n-n_lags,...,n+n1-n_lags
    for i in range(n - n_lags):
        X_train[i, :] = y_train[i : i + n_lags]
    for i in range(n1):
        if i < n_lags:
            X_test[i, :] = np.r_[y_train[n - n_lags + i :], y_test[:i]]
        else:
            X_test[i, :] = y_test[i - n_lags : i]
    y_train = y_train[n_lags:]
    return [X_train, X_test, y_train, y_test]


# Step 3) ################
def generate_bootstrap_samples(n, m, B, bootstrap_type, block_length, random_seed):
    samples_idx = np.zeros((B, m), dtype=int)

    for b in range(B):
        if bootstrap_type == "random":  # RB
            np.random.seed(b + random_seed)
            sample_idx = np.random.choice(a=n, size=m, replace=True)
        elif bootstrap_type == "nonoverlapping":  # NBB
            sample_idx = _id_nbb_bootstrap(
                n_obs=n, block_length=block_length, random_seed=b + random_seed,
            )
        elif bootstrap_type == "moving":  # MBB
            sample_idx = _id_mbb_bootstrap(
                n_obs=n, block_length=block_length, random_seed=b + random_seed,
            )
        elif bootstrap_type == "circular":  # CBB
            sample_idx = _id_cbb_bootstrap(
                n_obs=n, block_length=block_length, random_seed=b + random_seed,
            )
        elif bootstrap_type == "stationary":  # SBB
            sample_idx = _id_sbb_bootstrap(
                n_obs=n, block_length=block_length, random_seed=b + random_seed,
            )
        else:
            raise ValueError("boostrap_type not supported")      
        samples_idx[b, :] = sample_idx

    return samples_idx


# Non-overlapping block bootstrap (NBB)
def _id_nbb_bootstrap(
    n_obs: int, block_length: int, random_seed: Optional[int] = 10
) -> np.ndarray:
    """Create bootstrapped indexes with the none overlapping block bootstrap
    ('nbb') strategy given the number of observations in a timeseries and
    the length of the blocks.
    Returns
    -------
    _id : array
        Bootstrapped indexes.
    """
    rng = default_rng(random_seed)
    
    n_blocks = int(np.ceil(n_obs / block_length))
    nexts = np.repeat([np.arange(0, block_length)], n_blocks, axis=0)
    
    blocks = rng.permutation(x=np.arange(0, n_obs, block_length)).reshape(-1, 1)
    _id = (blocks + nexts).ravel()[:n_obs]

    return _id


# Moving block bootstrap (MBB)
def _id_mbb_bootstrap(
    n_obs: int, block_length: int, random_seed: Optional[int]=10
) -> np.ndarray:
    """Create bootstrapped indexes with the moving block bootstrap
    ('mbb') strategy given the number of observations in a timeseries
    and the length of the blocks.
    Returns
    -------
    _id : array
        Bootstrapped indexes.
    """
    rng = default_rng(random_seed)
    try:
        rng_integers = rng.integers
    except AttributeError:
        rng_integers = rng.randint

    n_blocks = int(np.ceil(n_obs / block_length))
    nexts = np.repeat([np.arange(0, block_length)], n_blocks, axis=0)
    
    last_block = n_obs - block_length
    blocks = rng_integers(low=0, high=last_block, size=(n_blocks, 1), dtype=int)
    _id = (blocks + nexts).ravel()[:n_obs]

    return _id


# Circular block bootstrap (MBB)
def _id_cbb_bootstrap(
    n_obs: int, block_length: int, random_seed: Optional[int]=10
) -> np.ndarray:
    """Create bootstrapped indexes with the circular block bootstrap
    ('cbb') strategy given the number of observations in a timeseries
    and the length of the blocks.
    Returns
    -------
    _id : array
        Bootstrapped indexes.
    """
    rng = default_rng(random_seed)
    try:
        rng_integers = rng.integers
    except AttributeError:
        rng_integers = rng.randint

    n_blocks = int(np.ceil(n_obs / block_length))
    nexts = np.repeat([np.arange(0, block_length)], n_blocks, axis=0)
    
    last_block = n_obs
    blocks = rng_integers(low=0, high=last_block, size=(n_blocks, 1), dtype=int)
    _id = np.mod((blocks + nexts).ravel(), n_obs)[:n_obs]

    return _id


# Stationary block bootstrap (SBB)
def _id_sbb_bootstrap(
    n_obs: int, block_length: int, random_seed: Optional[int]=10
) -> np.ndarray:
    """Create bootstrapped indexes with the stationary block bootstrap
    ('sb') strategy given the number of observations in a timeseries
    and the length of the blocks.
    Returns
    -------
    _id : array
        Bootstrapped indexes.
    """
    rng = default_rng(random_seed)
    try:
        rng_integers = rng.integers
    except AttributeError:
        rng_integers = rng.randint
    #
    rng_poisson = rng.poisson
    #
    random_block_length = rng_poisson(lam=block_length, size=n_obs)
    random_block_length[random_block_length < 3] = 3
    random_block_length[random_block_length >= n_obs] = n_obs
    random_block_length = random_block_length[random_block_length.cumsum() <= n_obs]
    residual_block = n_obs - random_block_length.sum()
    if residual_block > 0:
        random_block_length = np.append(random_block_length, residual_block)
    #
    n_blocks = random_block_length.shape[0]
    nexts = np.zeros((n_blocks, random_block_length.max() + 1))
    nexts[np.arange(n_blocks), random_block_length] = 1
    nexts = np.flip(nexts, 1).cumsum(1).cumsum(1).ravel()
    nexts = (nexts[nexts > 1] - 2).astype(int)
    #
    last_block = n_obs - random_block_length.max()
    blocks = np.zeros(n_obs, dtype=int)
    if last_block > 0:
        blocks = rng_integers(low=0, high=last_block, size=n_blocks)
        blocks = np.repeat(blocks, random_block_length)
    _id = blocks + nexts
    #
    return _id


# Step 4) ################
def initiate_regressor(model_type):
    min_alpha = 0.0001
    max_alpha = 10

    if model_type == "lasso":
        model = LassoCV(alphas=np.linspace(min_alpha, max_alpha, 10))
    elif model_type == "ridge":
        model = RidgeCV(alphas=np.linspace(min_alpha, max_alpha, 10))
    elif model_type == "random_forest":
        model = RandomForestRegressor(
            n_estimators=10,
            criterion="squared_error",
            bootstrap=False,
            max_depth=2,
            n_jobs=-1,
        )
    elif model_type == "extra_trees":
        model = ExtraTreesRegressor(
            n_estimators=10,
            criterion="squared_error",
            bootstrap=False,
            max_depth=2,
            n_jobs=-1
        )
    else:
        raise ValueError("model_type not supported")

    return model

In [71]:
#Decomposing Seasonal and trend components
def decompose(data, type_="stl"):
    if type_=="stl":
        stl_model=STL(data, period=24, seasonal=7)
        res=stl_model.fit()
        seasonal=res.seasonal
        trend=res.trend
        residual=res.resid
    return seasonal, trend, residual

#Restore Seasonal and trend components
def restore(residual, seasonal, trend, type_="add"):
    if type_=="add":
        yt=residual+seasonal+trend
    return yt

#Inverse Transform Yeojohnson Function
def invert_yeojhonson(value, lmbda):
    if value>= 0 and lmbda == 0:
        return exp(value) - 1
    elif value >= 0 and lmbda != 0:
        return (value * lmbda + 1) ** (1 / lmbda) - 1
    elif value < 0 and lmbda != 2:
        return 1 - (-(2 - lmbda) * value + 1) ** (1 / (2 - lmbda))
    elif value < 0 and lmbda == 2:
        return 1 - exp(-value)

#Inverse Transform Function (in general)
def inverse_transform(data, lmbda, type_="yeojohnson"):
    if type_=="yeojohnson":
        inverted = [invert_yeojhonson(x, lmbda) for x in data]
        inverted_array=np.array(inverted)
    elif type_=="boxcox":
        inverted_array=inv_boxcox(data, lmbda)
    return inverted_array

In [123]:
# Main executable function (yeojohnson and STL)

def enbpi_exec(
    data_list,
    max_data_size,
    method,
    train_fractions,
    B_list,
    bootstrap_types,
    block_lengths,
    model_types,
    alphas,
    itrial=0,
    one_dim_list=[False],
    n_lags=1,
    output_name = "many_data",
    verbose=1,
):

    # Step 1) Extract and Clean Data
    for data_name in data_list:
        if verbose >= 1: 
            print("\n\n" + "="*80)
            print(f"data_name='{data_name}'")
        data_df, target_column = read_data(data_name, max_data_size)
        tab_str = "- "
        if verbose == 2:
            print(f"{tab_str}data_name shape:{data_df.shape}")
            print(f"{tab_str}target_column='{target_column}'")

        # STEP 2): Preprocess Data ################    
        for one_dim in one_dim_list:
            if verbose >= 1: print(f"\n{tab_str}one_dim={one_dim}")
            # Separate results output for one_dim True and False
            results_df = pd.DataFrame()
            # results_df = pd.DataFrame(columns=_get_results_columns())
            for train_fraction in train_fractions:
                tab_str = "  " + tab_str
                if verbose >= 1: print(f"\n{tab_str}train_fraction={train_fraction}")
                # 2a) Split train/test sets
                # get X and y data
                data_X = data_df.drop(target_column, axis=1)
                data_y = data_df[target_column]
                # get train idx to split train and test
                train_size = int(len(data_df) * train_fraction)
                # split into 4
                X_train = data_X.iloc[:train_size, :]
                X_test = data_X.iloc[train_size:, :]
                y_train = data_y[:train_size]
                y_test = data_y[train_size:]

                # convert X and y test data to numpy
                X_test = X_test.to_numpy().copy()
                y_test = y_test.to_numpy().copy()
                
                # 2b) AR transformation (deleted for now)
                
                # 2c) Data Transformation and Seasonal/Trend Decomposition
                # X Variables
                cols=X_train.columns
                lmbda_dict={}
                xt_dict={}
                season_dict={}
                trend_dict={}
                resid_dict={}
                for col in cols:
                    # Transformation
                    pt = PowerTransformer() #Yeo-johnson
                    pt.fit(np.array(X_train[col]).reshape(-1,1))
                    lmbda=pt.lambdas_
                    xt=pt.transform(np.array(X_train[col]).reshape(-1,1))
                    lmbda_dict.update({col:lmbda})
                    xt_dict.update({col:xt})

                    # STL decompose
                    seasonal, trend, residual=decompose(xt.flatten(), "stl")
                    season_dict.update({col:seasonal})
                    trend_dict.update({col:trend})
                    resid_dict.update({col:residual})
                X_train_new=pd.DataFrame(resid_dict)
                # Target column Y
                pt.fit(np.array(y_train).reshape(-1,1))
                y_lmbda=pt.lambdas_
                yt=pt.transform(np.array(y_train).reshape(-1,1))
                seasonal_y, trend_y, y_train_new=decompose(yt.flatten(), "stl")
                # STEP 3) Bootstrap Synthetic Data ################
                for B in B_list:
                    for bootstrap_type in bootstrap_types:
                        for block_length in block_lengths:
                            if verbose >= 1:
                                print(f"{tab_str}B={B}, bootstrap_type='{bootstrap_type}', block_length={block_length}")
                            n = len(X_train)
                            n1 = len(X_test)
                            m = n
                            # tot_trial = 1    # For CP method that randomizes (for EnbPI, should be set to 1)
                            itrial = itrial  # for tot_trial in range(tot_trial)
                            random_seed = B + itrial
                            boot_samples_idx = generate_bootstrap_samples(
                                n=n,
                                m=m,
                                B=B,
                                bootstrap_type=bootstrap_type,
                                block_length=block_length,
                                random_seed=random_seed,
                            )
                            tab_str = "  " + tab_str
                            if verbose == 2: print(f"{tab_str}Bootstrap samples indexes shape: {boot_samples_idx.shape}")
                            # print(boot_samples_idx)

                            # STEP 4) Train Bootstrapped Models ################
                            # tab_str = "  " + tab_str    
                            for model_type in model_types:
                                # if verbose >= 1: print(f"{tab_str}model_type='{model_type}'")
                                # hold predictions from each f^b
                                boot_predictions = np.zeros((B, (n + n1)), dtype=float)
                                # for i^th column, it shows which f^b uses i in training (so exclude in aggregation)
                                in_boot_sample = np.zeros((B, n), dtype=bool)
                                for b in range(B):
                                    X_train_boot = X_train_new.iloc[boot_samples_idx[b], :]
                                    y_train_boot = y_train_new[boot_samples_idx[b],]
                                    # Reconstruct and inverse transform
                                    x_restored_dict={}
                                    x_original_dict={}
                                    # X variables
                                    for col in cols:
                                        # Add back seasonal and trend components
                                        x_restored=X_train_boot[col]+season_dict[col]+trend_dict[col]
                                        x_restored_dict.update({col:x_restored})
                                        # Inverse transformation
                                        pt.fit(np.array(X_train[col]).reshape(-1,1))
                                        x_original=pt.inverse_transform(np.array(x_restored).reshape(-1,1))
                                        x_original_dict.update({col:x_original.flatten()})
                                    x_original_df=pd.DataFrame(x_original_dict)
                                    if x_original_df.isnull().sum().sum()>0:
                                        x_original_df.fillna(method='ffill', inplace=True)
                                    X_train_boot_o=x_original_df.to_numpy()
                                    # Target column
                                    pt.fit(np.array(y_train).reshape(-1,1))
                                    y_restored=y_train_boot+seasonal_y+trend_y
                                    y_train_boot_o=np.array(pt.inverse_transform(np.array(y_restored).reshape(-1,1)))
    
                                    model = initiate_regressor(model_type)
                                    model.fit(
                                        X_train_boot_o,
                                        y_train_boot_o,
                                        # epochs=10,
                                        # batch_size=bsize,
                                        # callbacks=[callback],
                                        # verbose=0,
                                    )
                                    # Model's prediction for every observation on every bootstrapped data
                                    boot_predictions[b] = model.predict(np.r_[X_train, X_test]).flatten()
                                    # Whether a observation in the training data is sampled for the given bootstrapped data
                                    # If it is true, then this observation should NOT be used for model prediction
                                    in_boot_sample[b, boot_samples_idx[b]] = True
                                tab_str = "  " + tab_str
                                if verbose == 2:
                                    print(f"{tab_str}boot_predictions shape: {boot_predictions.shape}")
                                    print(f"{tab_str}in_boot_sample shape: {in_boot_sample.shape}")

                                # STEP 5) LOO Predict and Mean Aggregate
                                y_in_pred = np.zeros(n)          # Mean aggregated predict on in-sample train set
                                y_LOO_preds = np.zeros((n, n1))  # Non-aggregated predict on OOS test set
                                y_pred = np.zeros(n1)            # Aggregated predict on OOS test set

                                ensemble_online_resid = np.array([])    # LOO scores
                                # LOO in-sample train predict and OOS test predict 
                                for i in range(n):
                                    b_keep = np.argwhere(~(in_boot_sample[:, i])).reshape(-1)
                                    if len(b_keep) > 0:
                                        # Get the LOO predictions for i for both in-sample and OOS
                                        y_train_LOO_boot_preds = boot_predictions[b_keep, i]
                                        y_in_pred[i] = y_train_LOO_boot_preds.mean()  # Mean aggregate to form a scalar value
                                        y_LOO_boot_preds = boot_predictions[b_keep, n:]
                                        y_LOO_preds[i] = y_LOO_boot_preds.mean(0)
                                    else:  # if aggregating an empty set of models, predict zero everywhere
                                        if verbose == 2: print(f"{tab_str}  WARNING: no bootstrapped models available for i={i}")
                                y_pred = y_LOO_preds.mean(0)
                                if verbose == 2:
                                    print(f"{tab_str}y_in_pred shape: {y_in_pred.shape}")
                                    print(f"{tab_str}y_pred shape: {y_pred.shape}")
                                    print(f"{tab_str}  y_LOO_preds shape: {y_LOO_preds.shape}")

                                # STEP 6) Aggregate Model with LOO
                                for alpha in alphas:
                                    # if verbose >= 1: print(f"{tab_str}alpha={alpha}")
                                    # Get absolute residual and sort
                                    abs_resid = np.abs(np.subtract(y_train.to_numpy(), y_in_pred))
                                    abs_resid.sort()
                                    # Get q-hat value
                                    q_idx = int(-len(abs_resid)*alpha)
                                    q_hat = abs_resid[q_idx]
                                    # Evaluation metric 1: PI width
                                    width = round(q_hat*2, 6)
                                    tab_str = "  " + tab_str
                                    if verbose == 2: print(f"{tab_str}width={width}")

                                    # STEP 7) Get Coverage Rate from Test
                                    # Get OOS absolute residual and sort
                                    oos_abs_resid = np.abs(np.subtract(y_test, y_pred))
                                    oos_abs_resid.sort()
                                    # Count number of residuals below or equal to q_hat, or within the interval width.
                                    n_coverage = np.asarray(oos_abs_resid <= q_hat).sum()
                                    # Evaluation metric 2: Coverage rate
                                    coverage = round(n_coverage/n1, 6)
                                    if verbose == 2: print(f"{tab_str}coverage={coverage}")
                                    """
                                    NOTE: I am using the symmetical conformal prediction (like in NP), instead of the
                                          fancy method that Chen did. That is why the coverage rate is near the 
                                          theroretical guarantee limit of 2*alpha.
                                    """

                                    #### STEP 8) Output Results
                                    results_row = {
                                        "data_name": data_name,
                                        "one_dim": one_dim,
                                        "itrial": itrial,
                                        "train_fraction": train_fraction,
                                        "B": B,
                                        "bootstrap_type": bootstrap_type,
                                        "block_length": block_length,
                                        "model_type": model.__class__.__name__,  # "muh_fun"
                                        "method": method,
                                        "alpha": alpha,
                                        "width": width,    
                                        "coverage": coverage,
                                    }
                                    results_df = pd.concat([results_df, pd.DataFrame([results_row])], ignore_index=True)

                                    tab_str = tab_str[2:]
                                tab_str = tab_str[2:]
                            tab_str = tab_str[2:]
                tab_str = tab_str[4:]

    results_df.to_csv(f'Results/{output_name}_many_alpha_many_train_revamp.csv', index=False)

In [25]:
# Params
# Step 1) params
# cali_cities = ["Palo_Alto"]
# data_list = _get_cali_cities()
# data_list = ["Solar_Atl"] + cali_cities + ["appliances", "Beijing_air"]
data_list = ["Solar_Atl"]
max_data_size = 10000

# Step 2) params
train_fractions = [0.8001]
# train_fractions = [0.1, 0.2]
# train_fractions = [0.5, 0.6, 0.7, 0.8]
# train_fractions = [0.3, 0.4, 0.5, 0.6, 0.7]  # EnbPI WP Pg. 8
# print(train_fractions)
# one_dim_list = [True]
one_dim_list = [False]
n_lags = 24  # 'n_lags' aka 'd` is num_lookbacks for AR-transformer

# Step 3) params
itrial = 0
B_list = [30]  # number of bootstraps list
# B_list = [20, 25, 30, 35, 40, 45, 50]  # EnbPI WP Pg. 11: between 20-50 is sufficient
# bootstrap_types = ["random", "moving"]  # "nonoverlapping" has problem -> IndexError: index 851 is out of bounds for axis 0 with size 851
# bootstrap_types = ["random", "moving", "circular", "stationary"]
bootstrap_types = ["random", "moving", "nonoverlapping"]
block_lengths = [6, 12, 24]
# block_lengths = [6, 12, 24, 48, 60, 72]

# Step 4) params
# model_types = ["ridge", "random_forest"]  # muh_fun
model_types = ["lasso", "ridge", "random_forest"]  # "neural_networks"
method = "Ensemble"

# Step 6) params
alphas = [0.05, 0.1, 0.15]
# alphas = [0.05, 0.1, 0.15, 0.2, 0.25]  # EnbPI WP Pg. 17: five equally spaces 1-a [0.75, 0.95] are chosen
# alphas = [0.01, 0.10, 0.20, 0.50, 0.80, 0.90, 0.99]
# alphas = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.50, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99]

# Step 8) params
output_name = "Solar_Atl_Yeo_STL"
verbose = 1  # 0 no print statement, 1 only params, 2 everything

In [124]:
%%time
# Execute!

enbpi_exec(
    # Step 1) Extract and Clean Data
    data_list=data_list,
    max_data_size=max_data_size,
    # Step 2) Preprocess Data
    train_fractions=train_fractions,
    one_dim_list=one_dim_list,
    n_lags=n_lags,
    # Step 3) Preprocess Data
    itrial=itrial,
    B_list=B_list,
    bootstrap_types=bootstrap_types,
    block_lengths=block_lengths,
    # STEP 4) Train Bootstrapped Models
    model_types=model_types,
    method=method,
    # STEP 6) Get Conformal Prediction Interval
    alphas=alphas,
    # STEP 8) Output Results
    output_name=output_name,
    verbose=verbose,
)



data_name='Solar_Atl'

- one_dim=False

  - train_fraction=0.8001
  - B=30, bootstrap_type='random', block_length=6


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22846606.139671154, tolerance: 7349.412577414882
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3851349.01786457, tolerance: 6789.0880515263825
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23268546.999920115, tolerance: 7316.801279367802
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 83830.09326028079, tolerance: 6845.107862470016
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22873188.59890814, tolerance: 7135.818077855921
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15456347.770944878, tolerance: 7135.818077855921
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23618466.94671875, tolerance: 7515.129723968868
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 23367051.92726619, tolerance: 7028.7744354823935
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

...

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9140.238210380077, tolerance: 8644.860301056571
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 18271.584542900324, tolerance: 8644.860301056571
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5922394.72049927, tolerance: 8417.201791779584
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 27570227.653970867, tolerance: 8417.201791779584
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 29282252.208544713, tolerance: 8754.387086384451
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25928112.394796122, tolerance: 8077.370665098837
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/li

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vec

CPU times: user 3min 37s, sys: 2.32 s, total: 3min 39s
Wall time: 4min 19s


In [2]:
pd.read_csv(f'Results/Solar_Atl_original_many_alpha_many_train_revamp.csv')

,data_name,one_dim,itrial,train_fraction,B,bootstrap_type,block_length,model_type,method,alpha,width,coverage
0,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.05,432.118059,0.995431
1,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.10,293.483742,0.972016
2,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.15,240.171872,0.948601
3,Solar_Atl,False,0,0.8001,30,random,6,RidgeCV,Ensemble,0.05,431.955457,0.995431
4,Solar_Atl,False,0,0.8001,30,random,6,RidgeCV,Ensemble,0.10,293.525239,0.972016
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RidgeCV,Ensemble,0.10,532.000000,0.994289
77,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RidgeCV,Ensemble,0.15,344.000000,0.950885
78,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RandomForestRegressor,Ensemble,0.05,736.000000,1.000000
79,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RandomForestRegressor,Ensemble,0.10,532.000000,0.994289


In [126]:
pd.read_csv(f'Results/Solar_Atl_Yeo_STL_many_alpha_many_train_revamp.csv')

,data_name,one_dim,itrial,train_fraction,B,bootstrap_type,block_length,model_type,method,alpha,width,coverage
0,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.05,4.537972e+02,0.998287
1,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.10,3.107699e+02,0.977727
2,Solar_Atl,False,0,0.8001,30,random,6,LassoCV,Ensemble,0.15,2.369464e+02,0.944603
3,Solar_Atl,False,0,0.8001,30,random,6,RidgeCV,Ensemble,0.05,1.113008e+26,1.000000
4,Solar_Atl,False,0,0.8001,30,random,6,RidgeCV,Ensemble,0.10,1.001708e+26,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RidgeCV,Ensemble,0.10,5.320000e+02,0.994289
77,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RidgeCV,Ensemble,0.15,3.440000e+02,0.950885
78,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RandomForestRegressor,Ensemble,0.05,7.360000e+02,1.000000
79,Solar_Atl,False,0,0.8001,30,nonoverlapping,24,RandomForestRegressor,Ensemble,0.10,5.320000e+02,0.994289


# Test code

In [90]:
data_df, target_column = read_data("Solar_Atl", max_data_size)
data_X = data_df.drop(target_column, axis=1)
data_y = data_df[target_column]
# get train idx to split train and test
train_size = int(len(data_df) * 0.8001)
# split into 4
X_train = data_X.iloc[:train_size, :]
X_test = data_X.iloc[train_size:, :]
y_train = data_y[:train_size]
y_test = data_y[train_size:]

# convert X and y test data to numpy
X_test = X_test.to_numpy().copy()
y_test = y_test.to_numpy().copy()

# 2c) Data Transformation and Seasonal/Trend Decomposition
# X Variables
cols=X_train.columns
lmbda_dict={}
xt_dict={}
season_dict={}
trend_dict={}
resid_dict={}
for col in cols:
    # Transformation
    pt = PowerTransformer() #Yeo-johnson
    pt.fit(np.array(X_train[col]).reshape(-1,1))
    lmbda=pt.lambdas_
    xt=pt.transform(np.array(X_train[col]).reshape(-1,1))
    lmbda_dict.update({col:lmbda})
    xt_dict.update({col:xt})

    # STL decompose
    seasonal, trend, residual=decompose(xt.flatten(), "stl")
    season_dict.update({col:seasonal})
    trend_dict.update({col:trend})
    resid_dict.update({col:residual})
X_train_new=pd.DataFrame(resid_dict)
# Target column Y
pt.fit(np.array(y_train).reshape(-1,1))
y_lmbda=pt.lambdas_
yt=pt.transform(np.array(y_train).reshape(-1,1))
seasonal_y, trend_y, y_train_new=decompose(yt.flatten(), "stl")

In [97]:
n = len(X_train)
n1 = len(X_test)
m = n
# tot_trial = 1    # For CP method that randomizes (for EnbPI, should be set to 1)
itrial = itrial  # for tot_trial in range(tot_trial)
B=30
random_seed = B + itrial
bootstrap_type='random'
boot_samples_idx = generate_bootstrap_samples(
    n=n,
    m=m,
    B=B,
    bootstrap_type=bootstrap_type,
    block_length=block_length,
    random_seed=random_seed,
)


# STEP 4) Train Bootstrapped Models ################
b=0
boot_predictions = np.zeros((B, (n + n1)), dtype=float)
# for i^th column, it shows which f^b uses i in training (so exclude in aggregation)
in_boot_sample = np.zeros((B, n), dtype=bool)

X_train_boot = X_train_new.iloc[boot_samples_idx[b], :]
y_train_boot = y_train_new[boot_samples_idx[b],]
# Reconstruct and inverse transform
x_restored_dict={}
x_original_dict={}
# X variables
for col in cols:
    # Add back seasonal and trend components
    x_restored=X_train_boot[col]+season_dict[col]+trend_dict[col]
    x_restored_dict.update({col:x_restored})
    # Inverse transformation
    pt.fit(np.array(X_train[col]).reshape(-1,1))
    x_original=pt.inverse_transform(np.array(x_restored).reshape(-1,1))
    x_original_dict.update({col:x_original.flatten()})
X_train_boot_o=pd.DataFrame(x_original_dict).to_numpy()
# Target column
pt.fit(np.array(y_train).reshape(-1,1))
y_restored=y_train_boot+seasonal_y+trend_y
y_train_boot_o=np.array(pt.inverse_transform(np.array(y_restored).reshape(-1,1)))


In [113]:
pd.DataFrame(x_original_dict)

,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
0,-0.228475,-13.206782,0.109733,3.086556,57.217888,-5.254085,989.044352
1,-0.283025,-11.496401,0.109743,2.769974,61.070831,-4.646910,990.223794
2,0.071216,-14.326029,0.109776,1.525295,55.663233,-6.290479,990.048034
3,-0.297287,-16.209411,0.108868,2.092535,61.101774,-8.719054,990.896295
4,-0.279004,-11.363102,0.109852,1.838410,61.498508,-6.257962,991.265523
...,...,...,...,...,...,...,...
7003,-0.207010,19.975799,0.130272,0.653950,97.754012,20.440978,980.047932
7004,-0.412309,15.933447,0.130224,0.660292,95.617706,16.705656,983.418787
7005,-0.314473,16.205772,0.130212,0.708755,99.230293,15.509841,981.971542
7006,-0.361495,16.945975,0.129907,0.625927,99.879635,16.617634,982.924930


In [116]:
df_atl.drop('DHI', axis=1)[:7008]

,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
0,0,-12.3,0.11,1.9,59.45,-5.7,989
1,0,-12.6,0.11,1.8,59.64,-6.0,989
2,0,-12.7,0.11,1.8,60.25,-6.2,990
3,0,-12.8,0.11,1.9,60.54,-6.4,990
4,0,-13.2,0.11,1.9,60.17,-6.8,991
...,...,...,...,...,...,...,...
7003,0,16.9,0.13,0.6,100.00,16.9,983
7004,0,16.8,0.13,0.6,100.00,16.8,983
7005,0,16.9,0.13,0.6,100.00,16.9,982
7006,0,17.0,0.13,0.6,100.00,17.0,982


In [109]:
pd.DataFrame(x_original_dict).isnull().sum().sum()

42

In [112]:
for col in cols:
    print(pd.DataFrame(x_original_dict)[col].isnull().sum())

26
0
1
0
15
0
0


In [102]:
model = initiate_regressor("lasso")
model.fit(
    X_train_boot_o,
    y_train_boot_o,
    # epochs=10,
    # batch_size=bsize,
    # callbacks=[callback],
    # verbose=0,
)

ValueError: Input X contains NaN.
LassoCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [118]:
pd.DataFrame(x_original_dict).fillna(method='ffill').isnull().sum().sum()

0

In [119]:
n = len(X_train)
n1 = len(X_test)
m = n
# tot_trial = 1    # For CP method that randomizes (for EnbPI, should be set to 1)
itrial = itrial  # for tot_trial in range(tot_trial)
B=30
random_seed = B + itrial
bootstrap_type='random'
boot_samples_idx = generate_bootstrap_samples(
    n=n,
    m=m,
    B=B,
    bootstrap_type=bootstrap_type,
    block_length=block_length,
    random_seed=random_seed,
)


# STEP 4) Train Bootstrapped Models ################
b=0
boot_predictions = np.zeros((B, (n + n1)), dtype=float)
# for i^th column, it shows which f^b uses i in training (so exclude in aggregation)
in_boot_sample = np.zeros((B, n), dtype=bool)

X_train_boot = X_train_new.iloc[boot_samples_idx[b], :]
y_train_boot = y_train_new[boot_samples_idx[b],]
# Reconstruct and inverse transform
x_restored_dict={}
x_original_dict={}
# X variables
for col in cols:
    # Add back seasonal and trend components
    x_restored=X_train_boot[col]+season_dict[col]+trend_dict[col]
    x_restored_dict.update({col:x_restored})
    # Inverse transformation
    pt.fit(np.array(X_train[col]).reshape(-1,1))
    x_original=pt.inverse_transform(np.array(x_restored).reshape(-1,1))
    x_original_dict.update({col:x_original.flatten()})
x_original_df=pd.DataFrame(x_original_dict)
if x_original_df.isnull().sum().sum()>0:
    x_original_df.fillna(method='ffill', inplace=True)
X_train_boot_o=x_original_df.to_numpy()
# Target column
pt.fit(np.array(y_train).reshape(-1,1))
y_restored=y_train_boot+seasonal_y+trend_y
y_train_boot_o=np.array(pt.inverse_transform(np.array(y_restored).reshape(-1,1)))

In [120]:
model = initiate_regressor("lasso")
model.fit(
    X_train_boot_o,
    y_train_boot_o,
    # epochs=10,
    # batch_size=bsize,
    # callbacks=[callback],
    # verbose=0,
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:1563: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 22846606.139671154, tolerance: 7349.412577414882
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:614: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3851349.01786457, tolerance: 6789.0880515263825
  model = cd_fast.enet_coordinate_descent_gram(
/home/ec2-user/anaconda3/envs/python3/lib

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.774e+07, tolerance: 8.128e+03
  model = cd_fast.enet_coordinate_descent(


LassoCV(alphas=array([1.0000e-04, 1.1112e+00, 2.2223e+00, 3.3334e+00, 4.4445e+00,
       5.5556e+00, 6.6667e+00, 7.7778e+00, 8.8889e+00, 1.0000e+01]))

# Box-cox transformation and STL

In [4]:
df_atl, target_solar=read_data('Solar_Atl', 10000)

In [18]:
df_atl

,DHI,DNI,Dew Point,Surface Albedo,Wind Speed,Relative Humidity,Temperature,Pressure
0,0.0,0,-12.3,0.11,1.9,59.45,-5.7,989
1,0.0,0,-12.6,0.11,1.8,59.64,-6.0,989
2,0.0,0,-12.7,0.11,1.8,60.25,-6.2,990
3,0.0,0,-12.8,0.11,1.9,60.54,-6.4,990
4,0.0,0,-13.2,0.11,1.9,60.17,-6.8,991
...,...,...,...,...,...,...,...,...
8754,0.0,0,17.7,0.13,2.4,95.70,18.4,979
8755,0.0,0,-7.3,0.11,1.0,73.64,-3.3,987
8756,0.0,0,-8.3,0.11,1.5,71.91,-4.0,987
8757,0.0,0,-9.9,0.11,1.8,66.29,-4.6,987


In [22]:
#Series with positive values
df_atl['Relative Humidity']

0       59.45
1       59.64
2       60.25
3       60.54
4       60.17
        ...  
8754    95.70
8755    73.64
8756    71.91
8757    66.29
8758    63.41
Name: Relative Humidity, Length: 8759, dtype: float64

In [12]:
#Box-cox Transformation
yt, lmbda=stats.boxcox(df_atl['Relative Humidity'])

In [13]:
#Transformed data
yt

array([ 8341.27543703,  8406.13388559,  8616.36349894, ...,
       13238.08359301, 10865.22635776,  9754.69962686])

In [14]:
#Optimal Lambda
lmbda

2.4272865602828544

In [15]:
#Seasonal and Trend Decomposition Using Loess
stl_model=STL(yt, period=24, seasonal=7)

In [16]:
res=stl_model.fit()

#Seasonal Component
seasonal=res.seasonal

#Trend Component
trend=res.trend

#Remainders
residual=res.resid

In [17]:
n=len(yt)
m=n
B=10
block_length=12
#Moving Block Boostrapping
boot_samples_idx = generate_bootstrap_samples(
                                n=n,
                                m=m,
                                B=B,
                                bootstrap_type='moving',
                                block_length=block_length,
                                random_seed=1,
                            )

In [18]:
#Bootstrapped samples index
boot_samples_idx

array([[4138, 4139, 4140, ..., 8090, 8091, 8092],
       [7326, 7327, 7328, ..., 5658, 5659, 5660],
       [7098, 7099, 7100, ..., 2389, 2390, 2391],
       ...,
       [6293, 6294, 6295, ..., 4325, 4326, 4327],
       [3687, 3688, 3689, ..., 4124, 4125, 4126],
       [6792, 6793, 6794, ..., 4985, 4986, 4987]])

In [19]:
#Reconstruct seasonal and trend components from the bootstrapped remainders
y_restored=residual[boot_samples_idx[0]]+seasonal+trend

In [21]:
y_restored

array([ 7770.8999581 ,  6585.80354131,  6062.45472222, ...,
       15729.65750671, 14736.93355328, 15818.09310832])

In [20]:
#Inverse Box-cox transformation
inv_boxcox(y_restored, lmbda)

array([57.74035493, 53.93554011, 52.12679136, ..., 77.20465696,
       75.15877652, 77.38318343])

# Yeo-johnson and STL

In [73]:
pt = PowerTransformer()
pt.fit(np.array(df_atl['Pressure']).reshape(-1,1))
pt.lambdas_

array([7.19838703])

In [31]:
df_atl['Pressure']

0       989
1       989
2       990
3       990
4       991
       ... 
8754    979
8755    987
8756    987
8757    987
8758    988
Name: Pressure, Length: 8759, dtype: int64

In [101]:
df_atl['DHI']

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8754    0.0
8755    0.0
8756    0.0
8757    0.0
8758    0.0
Name: DHI, Length: 8759, dtype: float64

In [66]:
df_atl.to_numpy()

array([[  0.  ,   0.  , -12.3 , ...,  59.45,  -5.7 , 989.  ],
       [  0.  ,   0.  , -12.6 , ...,  59.64,  -6.  , 989.  ],
       [  0.  ,   0.  , -12.7 , ...,  60.25,  -6.2 , 990.  ],
       ...,
       [  0.  ,   0.  ,  -8.3 , ...,  71.91,  -4.  , 987.  ],
       [  0.  ,   0.  ,  -9.9 , ...,  66.29,  -4.6 , 987.  ],
       [  0.  ,   0.  , -11.  , ...,  63.41,  -5.1 , 988.  ]])

In [74]:
yt=pt.transform(np.array(df_atl['Pressure']).reshape(-1,1))

In [75]:
yt

array([[1.44881582],
       [1.44881582],
       [1.64821772],
       ...,
       [1.05373917],
       [1.05373917],
       [1.25065848]])

In [76]:
pt.inverse_transform(yt)

array([[989.],
       [989.],
       [990.],
       ...,
       [987.],
       [987.],
       [988.]])

In [77]:
yt.flatten()

array([1.44881582, 1.44881582, 1.64821772, ..., 1.05373917, 1.05373917,
       1.25065848])

In [78]:
stl_model=STL(yt.flatten(), period=24, seasonal=7)

In [79]:
res=stl_model.fit()
seasonal=res.seasonal
trend=res.trend
residual=res.resid

In [80]:
n=len(yt)
m=n
B=10
block_length=12
boot_samples_idx = generate_bootstrap_samples(
                                n=n,
                                m=m,
                                B=B,
                                bootstrap_type='moving',
                                block_length=block_length,
                                random_seed=1,
                            )

In [81]:
y_restored=residual[boot_samples_idx[0]]+seasonal+trend

In [82]:
y_restored

array([1.61936452, 1.50071096, 1.74479886, ..., 0.7935808 , 0.75325744,
       0.84953127])

In [83]:
y_restored.reshape(-1,1)

array([[1.61936452],
       [1.50071096],
       [1.74479886],
       ...,
       [0.7935808 ],
       [0.75325744],
       [0.84953127]])

In [85]:
inverse_transform(y_restored.reshape(-1,1), pt.lambdas_)

/tmp/ipykernel_7301/667770755.py:32: RuntimeWarning: invalid value encountered in power
  return 1 - (-(2 - lmbda) * value + 1) ** (1 / (2 - lmbda))


array([[0.42277069],
       [0.4090281 ],
       [0.43645611],
       ...,
       [0.30277912],
       [0.29480333],
       [0.31336763]])

In [84]:
pt.inverse_transform(y_restored.reshape(-1,1))

array([[989.8556879 ],
       [989.260857  ],
       [990.48211453],
       ...,
       [985.66916378],
       [985.46189102],
       [985.95631769]])

In [90]:
sum(pt.inverse_transform(y_restored.reshape(-1,1))==np.nan)

array([0])

# Treatment 2 (Transform-STL-Reconstruct-Inverse Transform) 

In [ ]:
# Main executable function for treatment 2

def enbpi_exec_t2(
    data_list,
    max_data_size,
    method,
    train_fractions,
    B_list,
    bootstrap_types,
    block_lengths,
    model_types,
    alphas,
    itrial=0,
    n_lags=1,
    output_name = "many_data",
    verbose=1,
):

    # Step 1) Extract and Clean Data
    for data_name in data_list:
        if verbose >= 1: 
            print("\n\n" + "="*80)
            print(f"data_name='{data_name}'")
        data_df, target_column = read_data(data_name, max_data_size)
        tab_str = "- "
        if verbose == 2:
            print(f"{tab_str}data_name shape:{data_df.shape}")
            print(f"{tab_str}target_column='{target_column}'")

        # STEP 2): Preprocess Data ################    
        for one_dim in one_dim_list:
            if verbose >= 1: print(f"\n{tab_str}one_dim={one_dim}")
            # Separate results output for one_dim True and False
            results_df = pd.DataFrame()
            # results_df = pd.DataFrame(columns=_get_results_columns())
            for train_fraction in train_fractions:
                tab_str = "  " + tab_str
                if verbose >= 1: print(f"\n{tab_str}train_fraction={train_fraction}")
                # 2a) Split train/test sets
                # get X and y data
                data_X = data_df.drop(target_column, axis=1)
                data_y = data_df[target_column]
                # get train idx to split train and test
                train_size = int(len(data_df) * train_fraction)
                # split into 4
                X_train = data_X.iloc[:train_size, :]
                X_test = data_X.iloc[train_size:, :]
                y_train = data_y[:train_size]
                y_test = data_y[train_size:]

                # convert X and y test data to numpy
                X_test = X_test.to_numpy().copy()
                y_test = y_test.to_numpy().copy()
                
                # 2b) Yeojhonson Transformation and STL decomposition
                #X Variables
                cols=X_train.columns
                lmbda_dict={}
                xt_dict={}
                season_dict={}
                trend_dict={}
                resid_dict={}
                for col in cols:
                    #Yeojohnson transform
                    xt, lmbda=transform(X_train[col])
                    lmbda_dict.update({col:lmbda})
                    xt_dict.update({col:xt})

                    #STL decompose
                    seasonal, trend, residual=decompose(xt, "stl")
                    season_dict.update({col:seasonal})
                    trend_dict.update({col:trend})
                    resid_dict.update({col:residual})
                
                X_train_new=pd.DataFrame(resid_dict)
                #Target column Y
                yt, y_lmbda=transform(y_train)
                seasonal_y, trend_y, y_train_new=decompose(yt, "stl")

                # STEP 3) Bootstrap Synthetic Data ################
                for B in B_list:
                    for bootstrap_type in bootstrap_types:
                        for block_length in block_lengths:
                            if verbose >= 1:
                                print(f"{tab_str}B={B}, bootstrap_type='{bootstrap_type}', block_length={block_length}")
                            n = len(X_train)
                            n1 = len(X_test)
                            m = n
                            # tot_trial = 1    # For CP method that randomizes (for EnbPI, should be set to 1)
                            itrial = itrial  # for tot_trial in range(tot_trial)
                            random_seed = B + itrial
                            boot_samples_idx = generate_bootstrap_samples(
                                n=n,
                                m=m,
                                B=B,
                                bootstrap_type=bootstrap_type,
                                block_length=block_length,
                                random_seed=random_seed,
                            )
                            tab_str = "  " + tab_str
                            if verbose == 2: print(f"{tab_str}Bootstrap samples indexes shape: {boot_samples_idx.shape}")
                            # print(boot_samples_idx)

                            # STEP 4) Train Bootstrapped Models ################
                            # tab_str = "  " + tab_str    
                            for model_type in model_types:
                                # if verbose >= 1: print(f"{tab_str}model_type='{model_type}'")
                                # hold predictions from each f^b
                                boot_predictions = np.zeros((B, (n + n1)), dtype=float)
                                # for i^th column, it shows which f^b uses i in training (so exclude in aggregation)
                                in_boot_sample = np.zeros((B, n), dtype=bool)
                                for b in range(B):
                                    X_train_boot = X_train_new.iloc[boot_samples_idx[b], :]
                                    y_train_boot = y_train_new[boot_samples_idx[b],]
                                    #Reconstruct and inverse transform
                                    x_restored_dict={}
                                    x_original_dict={}
                                    #X variables
                                    for col in cols:
                                        #Add back seasonal and trend components
                                        x_restored=X_train_boot[col]+season_dict[col]+trend_dict[col]
                                        x_restored_dict.update({col:x_restored})
                                        #Inverse transformation
                                        x_original=inverse_transform(x_restored, lmbda_dict[col])
                                        x_original_dict.update({col:x_original})
                                    X_train_boot_o=pd.DataFrame(x_original_dict)
                                    #Target column
                                    y_restored=y_train_boot+seasonal_y+trend_y
                                    y_train_boot_o=inverse_transform(y_restored, y_lmbda)
    
                                    
                                    model = initiate_regressor(model_type)
                                    model.fit(
                                        #Convert the bootstrapped X and y bootstrapped data to numpy
                                        X_train_boot_o.to_numpy(),
                                        np.array(y_train_boot_o),
                                        # epochs=10,
                                        # batch_size=bsize,
                                        # callbacks=[callback],
                                        # verbose=0,
                                    )
                                    # Model's prediction for every observation on every bootstrapped data
                                    boot_predictions[b] = model.predict(np.r_[X_train.to_numpy(), X_test]).flatten()
                                    # Whether a observation in the training data is sampled for the given bootstrapped data
                                    # If it is true, then this observation should NOT be used for model prediction
                                    in_boot_sample[b, boot_samples_idx[b]] = True
                                tab_str = "  " + tab_str
                                if verbose == 2:
                                    print(f"{tab_str}boot_predictions shape: {boot_predictions.shape}")
                                    print(f"{tab_str}in_boot_sample shape: {in_boot_sample.shape}")

                                # STEP 5) LOO Predict and Mean Aggregate
                                y_in_pred = np.zeros(n)          # Mean aggregated predict on in-sample train set
                                y_LOO_preds = np.zeros((n, n1))  # Non-aggregated predict on OOS test set
                                y_pred = np.zeros(n1)            # Aggregated predict on OOS test set

                                ensemble_online_resid = np.array([])    # LOO scores
                                # LOO in-sample train predict and OOS test predict 
                                for i in range(n):
                                    b_keep = np.argwhere(~(in_boot_sample[:, i])).reshape(-1)
                                    if len(b_keep) > 0:
                                        # Get the LOO predictions for i for both in-sample and OOS
                                        y_train_LOO_boot_preds = boot_predictions[b_keep, i]
                                        y_in_pred[i] = y_train_LOO_boot_preds.mean()  # Mean aggregate to form a scalar value
                                        y_LOO_boot_preds = boot_predictions[b_keep, n:]
                                        y_LOO_preds[i] = y_LOO_boot_preds.mean(0)
                                    else:  # if aggregating an empty set of models, predict zero everywhere
                                        if verbose == 2: print(f"{tab_str}  WARNING: no bootstrapped models available for i={i}")
                                y_pred = y_LOO_preds.mean(0)
                                if verbose == 2:
                                    print(f"{tab_str}y_in_pred shape: {y_in_pred.shape}")
                                    print(f"{tab_str}y_pred shape: {y_pred.shape}")
                                    print(f"{tab_str}  y_LOO_preds shape: {y_LOO_preds.shape}")

                                # STEP 6) Aggregate Model with LOO
                                for alpha in alphas:
                                    # if verbose >= 1: print(f"{tab_str}alpha={alpha}")
                                    # Get absolute residual and sort
                                    abs_resid = np.abs(np.subtract(y_train, y_in_pred))
                                    abs_resid.sort()
                                    # Get q-hat value
                                    q_idx = int(-len(abs_resid)*alpha)
                                    q_hat = abs_resid[q_idx]
                                    # Evaluation metric 1: PI width
                                    width = round(q_hat*2, 6)
                                    tab_str = "  " + tab_str
                                    if verbose == 2: print(f"{tab_str}width={width}")

                                    # STEP 7) Get Coverage Rate from Test
                                    # Get OOS absolute residual and sort
                                    oos_abs_resid = np.abs(np.subtract(y_test, y_pred))
                                    oos_abs_resid.sort()
                                    # Count number of residuals below or equal to q_hat, or within the interval width.
                                    n_coverage = np.asarray(oos_abs_resid <= q_hat).sum()
                                    # Evaluation metric 2: Coverage rate
                                    coverage = round(n_coverage/n1, 6)
                                    if verbose == 2: print(f"{tab_str}coverage={coverage}")
                                    """
                                    NOTE: I am using the symmetical conformal prediction (like in NP), instead of the
                                          fancy method that Chen did. That is why the coverage rate is near the 
                                          theroretical guarantee limit of 2*alpha.
                                    """

                                    #### STEP 8) Output Results
                                    results_row = {
                                        "data_name": data_name,
                                        "one_dim": one_dim,
                                        "itrial": itrial,
                                        "train_fraction": train_fraction,
                                        "B": B,
                                        "bootstrap_type": bootstrap_type,
                                        "block_length": block_length,
                                        "model_type": model.__class__.__name__,  # "muh_fun"
                                        "method": method,
                                        "alpha": alpha,
                                        "width": width,    
                                        "coverage": coverage,
                                    }
                                    results_df = pd.concat([results_df, pd.DataFrame([results_row])], ignore_index=True)

                                    tab_str = tab_str[2:]
                                tab_str = tab_str[2:]
                            tab_str = tab_str[2:]
                tab_str = tab_str[4:]

    results_df.to_csv(f'Results/{output_name}_treatment2.csv', index=False)

In [ ]:
# Params
# Step 1) params
# cali_cities = ["Palo_Alto"]
# data_list = _get_cali_cities()
# data_list = ["Solar_Atl"] + cali_cities + ["appliances", "Beijing_air"]
data_list = ["Solar_Atl"]
max_data_size = 10000

# Step 2) params
train_fractions = [0.8001]
# train_fractions = [0.1, 0.2]
# train_fractions = [0.5, 0.6, 0.7, 0.8]
# train_fractions = [0.3, 0.4, 0.5, 0.6, 0.7]  # EnbPI WP Pg. 8
# print(train_fractions)
# one_dim_list = [True]
one_dim_list = [False]
n_lags = 24  # 'n_lags' aka 'd` is num_lookbacks for AR-transformer

# Step 3) params
itrial = 0
B_list = [30]  # number of bootstraps list
# B_list = [20, 25, 30, 35, 40, 45, 50]  # EnbPI WP Pg. 11: between 20-50 is sufficient
# bootstrap_types = ["random", "moving"]  # "nonoverlapping" has problem -> IndexError: index 851 is out of bounds for axis 0 with size 851
# bootstrap_types = ["random", "moving", "circular", "stationary"]
bootstrap_types = ["random", "moving", "nonoverlapping"]
block_lengths = [6, 12, 24]
# block_lengths = [6, 12, 24, 48, 60, 72]

# Step 4) params
# model_types = ["ridge", "random_forest"]  # muh_fun
model_types = ["lasso", "ridge", "random_forest"]  # "neural_networks"
method = "Ensemble"

# Step 6) params
alphas = [0.05, 0.1, 0.15]
# alphas = [0.05, 0.1, 0.15, 0.2, 0.25]  # EnbPI WP Pg. 17: five equally spaces 1-a [0.75, 0.95] are chosen
# alphas = [0.01, 0.10, 0.20, 0.50, 0.80, 0.90, 0.99]
# alphas = [0.01, 0.05, 0.10, 0.15, 0.20, 0.25, 0.50, 0.75, 0.80, 0.85, 0.90, 0.95, 0.99]

# Step 8) params
output_name = "Solar_Atl"
verbose = 1  # 0 no print statement, 1 only params, 2 everything

In [ ]:
%%time
# Execute!

enbpi_exec_t2(
    # Step 1) Extract and Clean Data
    data_list=data_list,
    max_data_size=max_data_size,
    # Step 2) Preprocess Data
    train_fractions=train_fractions,
    n_lags=n_lags,
    # Step 3) Preprocess Data
    itrial=itrial,
    B_list=B_list,
    bootstrap_types=bootstrap_types,
    block_lengths=block_lengths,
    # STEP 4) Train Bootstrapped Models
    model_types=model_types,
    method=method,
    # STEP 6) Get Conformal Prediction Interval
    alphas=alphas,
    # STEP 8) Output Results
    output_name=output_name,
    verbose=verbose,
)